# PEFT Finetuning for Neutralizing the Gender Biased Texts

This notebook uses int8 quantization and LoRA finetuning.

**_Note:_** To run this notebook on a machine with less than 24GB VRAM (e.g. T4 with 16GB) the context length of the training dataset needs to be adapted.
We do this based on the available VRAM during execution.
If you run into OOM issues try to further lower the value of train_config.context_length.

We run the notebook on a machine with GPU T4 $\times2$ accelarator.

### Step 0: Install pre-requirements and convert checkpoint

We need to have llama-recipes and its dependencies installed for this notebook, as well as HuggingFace evaluation metrics. Additionally, we need to log in with the huggingface_cli and make sure that the account is able to to access the Meta Llama weights.

In [1]:
! pip install --upgrade pip
! pip install evaluate
! pip install bert_score
! pip install hazm
! pip install llama-recipes ipywidgets

import huggingface_hub
huggingface_hub.login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 19.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 84.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 84.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: nltk
    Found existing ins

### Step 1: Load the model

Setup training configuration and load the model and tokenizer.

In [2]:
import torch
from transformers import LlamaForCausalLM, AutoTokenizer
from llama_recipes.configs import train_config as TRAIN_CONFIG

train_config = TRAIN_CONFIG()
train_config.model_name = "meta-llama/Meta-Llama-3-8B"
train_config.num_epochs = 3
train_config.run_validation = False
train_config.gradient_accumulation_steps = 4
train_config.batch_size_training = 1
train_config.lr = 2e-4
train_config.use_fast_kernels = True
train_config.use_fp16 = True
train_config.context_length = 1024 if torch.cuda.get_device_properties(0).total_memory < 16e9 else 2048 # T4 16GB or A10 24GB
train_config.batching_strategy = "packing"
train_config.output_dir = "meta-llama-sexbias"

from transformers import BitsAndBytesConfig
config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = LlamaForCausalLM.from_pretrained(
            train_config.model_name,
            device_map="auto",
            quantization_config=config,
            use_cache=False,
            attn_implementation="sdpa" if train_config.use_fast_kernels else None,
            torch_dtype=torch.float16,
        )

tokenizer = AutoTokenizer.from_pretrained(train_config.model_name)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Step 2: Test the Base Model

Here, we test the base model of `Llama3-8b` on the test set, by the evalation metrics of BLEU and BERTScore.

#### Prepare the Test Dataset

In [3]:
import datasets

# Load the test dataset
test_dataset = datasets.load_dataset("AmirMohammadFakhimi/gender_neutralize", split="test")

# Prepare the prompts for evaluation
prompt_template = (
        f"این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:\n{{dialog}}\n________\nمتن بدون بایاس جنسیتی:\n"
    )

def prepare_prompt(sample):
    return {
        "prompt": prompt_template.format(dialog=sample["dialogue"]),
        "summary": sample["summary"]
    }

test_dataset = test_dataset.map(prepare_prompt, remove_columns=list(test_dataset.features))

Generating train split:   0%|          | 0/1309 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/146 [00:00<?, ? examples/s]

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

#### Generate Predictions

In [8]:
def generate_predictions(model, tokenizer, dataset):
    predictions = []
    references = []

    model.eval()
    with torch.no_grad():
        for sample in dataset:
            model_input = tokenizer(sample["prompt"], return_tensors="pt").to("cuda")
            output = model.generate(**model_input, max_new_tokens=100)
            prediction = tokenizer.decode(output[0], skip_special_tokens=True)
        
            # Extract the part after "متن بدون بایاس جنسیتی:"
            prediction = prediction.split('\n________\nمتن بدون بایاس جنسیتی:\n')[1].strip()
            prediction = re.sub('متن بدون بایاس جنسیتی:', ' '  , prediction)
            # Remove non-Alphabetical characters and digits execpt dot
            prediction = re.sub(r'[^.آ-یA-Za-z\s]', ' ', prediction)
            # Define a regular expression pattern that matches one or more spaces
            pattern = re.compile(r" +")
            # Apply the pattern to the text and replace the matches with a single space
            prediction = pattern.sub(" ", prediction)
            
            predictions.append(prediction)
            references.append(sample["summary"])

    return predictions, references

# Generate predictions for base model
base_predictions, references = generate_predictions(model, tokenizer, test_dataset)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

#### Compute the Evaluation Metrics

In [9]:
from evaluate import load
from hazm import word_tokenize

bertscore_metric = load("bertscore")
bleu_metric = load('bleu')

def compute_metrics(predictions, references):
    # fix the references with the BLEU score HuggingFace format
    references = [[ref] for ref in references]

    # Compute BLEU score
    bleu_score = bleu_metric.compute(predictions=predictions, references=references, tokenizer=word_tokenize)#["bleu"]

    # Compute BERTScore
    bertscore = bertscore_metric.compute(predictions=predictions, references=references, lang="fa", model_type='microsoft/deberta-v2-xxlarge-mnli')
    bertscore_f1 = sum(bertscore["f1"]) / len(bertscore["f1"])

    return bleu_score, bertscore_f1

# Compute metrics for both models
base_bleu, base_bertscore = compute_metrics(base_predictions, references)

print(f"Base Model - BLEU: {base_bleu}, BERTScore F1: {base_bertscore}")

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

Base Model - BLEU: {'bleu': 0.19212972085620741, 'precisions': [0.2688223938223938, 0.2023511755877939, 0.17159916926272067, 0.145979492714517], 'brevity_penalty': 1.0, 'length_ratio': 2.5009052504526252, 'translation_length': 4144, 'reference_length': 1657}, BERTScore F1: 0.818024289730477


#### Sample Predictions
Run the base model on some example inputs:

In [3]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
مردان باید در برنامه‌ریزی برای تعطیلات خانوادگی پیش‌قدم باشند.
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
    
torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
2024-07-22 05:26:26.752843: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 05:26:26.752940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 05:26:26.906626: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
مردان باید در برنامه‌ریزی برای تعطیلات خانوادگی پیش‌قدم باشند.
________
متن بدون بایاس جنسیتی:
شما باید در برنامه‌ریزی برای تعطیلات خانوادگی پیش‌قدم باشید.



In [4]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
افراد ممکن است به جنبه های مختلف زندگی خود توجه کنند و تمرکز متفاوتی بر روی مهارت های حرفه ای داشته باشند.
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
    
torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
افراد ممکن است به جنبه های مختلف زندگی خود توجه کنند و تمرکز متفاوتی بر روی مهارت های حرفه ای داشته باشند.
________
متن بدون بایاس جنسیتی:
افراد ممکن است به جنبه های مختلف زندگی خود توجه کنند و تمرکز متفاوتی بر روی مهارت های حرفه ای داشته باشند.
________
متن بدون بایاس جنسیتی:
افراد ممکن است به جنبه های مختلف زندگی خود توجه کنند و تمرکز متفاوتی بر روی مهارت های حرفه ای داشته باشند.
________
متن بدون بایاس جنسیتی:
افراد ممکن است به جنبه های مختلف زندگی خود توجه کنند و تمرکز متفاوتی بر روی مه


In [5]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زنان باید سرپرست سازمان‌دهی رویدادهای اجتماعی باشند.
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
    
torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زنان باید سرپرست سازمان‌دهی رویدادهای اجتماعی باشند.
________
متن بدون بایاس جنسیتی:
زنان و مردان باید سرپرست سازمان‌دهی رویدادهای اجتماعی باشند.



In [5]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
المیرا اسم یک دختر است
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
المیرا اسم یک دختر است
________
متن بدون بایاس جنسیتی:
المیرا اسم یک دختر است
________
متن با بایاس جنسیتی:
المیرا اسم یک دختر است
________
متن بدون بایاس جنسیتی:
المیرا اسم یک دختر است
________
متن با بایاس جنسیتی:
المیرا اسم یک دختر است
________
متن بدون بایاس جنسیتی:
المیرا اسم یک دختر است
________
متن با بایاس جنسیتی:
المیرا اسم یک دختر است
________



In [8]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زن‌ها نمی‌توانند به خوبی مردها طراحی کنند.
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زن‌ها نمی‌توانند به خوبی مردها طراحی کنند.
________
متن بدون بایاس جنسیتی:
زن‌ها می‌توانند به خوبی مردها طراحی کنند.

متن با بایاس جنسیتی:
زن‌ها نمی‌توانند به خوبی مردها طراحی کنند.

متن با بایاس جنسیتی:
زن‌ها نمی‌توانند به خوبی مردها طراحی کنند.

متن با بایاس جنسیتی:
زن‌ها نمی‌توانند به خوبی مردها طراحی کنند.

متن با بایاس جنسیتی:
زن‌ها نمی‌توانند به خوبی مردها طراحی کنند.

متن با


In [9]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زن‌ها در ورزش‌های سنگین موفق نمی‌شوند
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زن‌ها در ورزش‌های سنگین موفق نمی‌شوند
________
متن بدون بایاس جنسیتی:
زن‌ها در ورزش‌های سنگین موفق نمی‌شوند



As you can see, the base model outputs are incorrect and sometimes, are invalid also; and get repeated weirdly.

### Step 3: Load the preprocessed dataset

We load and preprocess the samsum dataset which consists of curated pairs of dialogs and their summarization:

In [4]:
from dataclasses import dataclass
 
@dataclass
class samsum_dataset:
    dataset: str =  "samsum_dataset"
    train_split: str = "train"
    test_split: str = "validation"

In [5]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# This software may be used and distributed according to the terms of the Llama 3 Community License Agreement.

# For dataset details visit: https://huggingface.co/datasets/AmirMohammadFakhimi/gender_neutralize

import copy
import datasets

def get_preprocessed_samsum(dataset_config, tokenizer, split):
    dataset = datasets.load_dataset("AmirMohammadFakhimi/gender_neutralize", split=split)

    prompt = (
        f"این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:\n{{dialog}}\n________\nمتن بدون بایاس جنسیتی:\n"
    )

    def apply_prompt_template(sample):
        return {
            "prompt": prompt.format(dialog=sample["dialogue"]),
            "summary": sample["summary"],
        }

    dataset = dataset.map(apply_prompt_template, remove_columns=list(dataset.features))

    def tokenize_add_label(sample):
        prompt = tokenizer.encode(tokenizer.bos_token + sample["prompt"], add_special_tokens=False)
        summary = tokenizer.encode(sample["summary"] +  tokenizer.eos_token, add_special_tokens=False)

        sample = {
            "input_ids": prompt + summary,
            "attention_mask" : [1] * (len(prompt) + len(summary)),
            "labels": [-100] * len(prompt) + summary,
            }

        return sample

    dataset = dataset.map(tokenize_add_label, remove_columns=list(dataset.features))

    return dataset

In [6]:
def get_preprocessed_dataset(
    tokenizer, dataset_config, split: str = "train"
) -> torch.utils.data.Dataset:

    def get_split():
        return (
            dataset_config.train_split
            if split == "train"
            else dataset_config.test_split
        )

    return get_preprocessed_samsum(
        dataset_config,
        tokenizer,
        get_split(),
    )

In [7]:
from llama_recipes.data.concatenator import ConcatDataset
from llama_recipes.utils.config_utils import get_dataloader_kwargs

train_dataset = get_preprocessed_dataset(tokenizer, samsum_dataset, 'train')

train_dl_kwargs = get_dataloader_kwargs(train_config, train_dataset, tokenizer, "train")

if train_config.batching_strategy == "packing":
    train_dataset = ConcatDataset(train_dataset, chunk_size=train_config.context_length)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    num_workers=train_config.num_workers_dataloader,
    pin_memory=True,
    **train_dl_kwargs,
)

2024-07-22 03:58:38.292629: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 03:58:38.292732: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 03:58:38.418632: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Map:   0%|          | 0/1309 [00:00<?, ? examples/s]

Map:   0%|          | 0/1309 [00:00<?, ? examples/s]

Preprocessing dataset: 100%|██████████| 1309/1309 [00:00<00:00, 5204.91it/s]


### Step 4: Prepare model for PEFT

Let's prepare the model for Parameter Efficient Fine Tuning (PEFT):

In [8]:
from peft import get_peft_model, prepare_model_for_kbit_training, LoraConfig
from dataclasses import asdict
from llama_recipes.configs import lora_config as LORA_CONFIG

lora_config = LORA_CONFIG()
lora_config.r = 8
lora_config.lora_alpha = 32
lora_dropout: float=0.01

peft_config = LoraConfig(**asdict(lora_config))

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

### Step 5: Fine tune the model

Here, we fine tune the model for a single epoch.

In [9]:
import torch.optim as optim
from llama_recipes.utils.train_utils import train
from torch.optim.lr_scheduler import StepLR

model.train()

optimizer = optim.AdamW(
            model.parameters(),
            lr=train_config.lr,
            weight_decay=train_config.weight_decay,
        )
scheduler = StepLR(optimizer, step_size=1, gamma=train_config.gamma)

# Start the training process
results = train(
    model,
    train_dataloader,
    None,
    tokenizer,
    optimizer,
    scheduler,
    train_config.gradient_accumulation_steps,
    train_config,
    None,
    None,
    None,
    wandb_run=None,
)

/opt/conda/lib/python3.10/site-packages/torch/cuda/memory.py:330: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
Training Epoch: 1:   0%|          | 0/21 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicit

Max CUDA memory allocated was 9 GB
Max CUDA memory reserved was 9 GB
Peak active CUDA memory was 9 GB
CUDA Malloc retries : 0
CPU Total Peak Memory consumed during the train (max): 3 GB
Epoch 1: train_perplexity=1.1242, train_epoch_loss=0.1171, epoch time 598.1484485689999s


Training Epoch: 2:   0%|          | 0/21 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Training Epoch: 2/3, step 83/84 completed (loss: 0.017545850947499275): 100%|██████████| 21/21 [09:56<00:00, 28.41s/it]  


Max CUDA memory allocated was 9 GB
Max CUDA memory reserved was 9 GB
Peak active CUDA memory was 9 GB
CUDA Malloc retries : 0
CPU Total Peak Memory consumed during the train (max): 4 GB
Epoch 2: train_perplexity=1.0731, train_epoch_loss=0.0705, epoch time 597.295847465s


Training Epoch: 3:   0%|          | 0/21 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Training Epoch: 3/3, step 83/84 completed (loss: 0.015038029290735722): 100%|██████████| 21/21 [09:55<00:00, 28.37s/it]  


Max CUDA memory allocated was 9 GB
Max CUDA memory reserved was 9 GB
Peak active CUDA memory was 9 GB
CUDA Malloc retries : 0
CPU Total Peak Memory consumed during the train (max): 4 GB
Epoch 3: train_perplexity=1.0572, train_epoch_loss=0.0556, epoch time 596.5726425649998s


### Step 6:
Save model checkpoint

In [11]:
model.save_pretrained(train_config.output_dir)

#### Load the fine-tuned model (We recommend don't run this cell untill you get CUDA OutOfMemory Error)

In [2]:
from transformers import LlamaForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch

# Load the base model and tokenizer
base_model_name = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

# Use mixed precision and quantization to save memory
config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True  # Enable CPU offload to save GPU memory
)

base_model = LlamaForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    quantization_config=config,
    torch_dtype=torch.float16
)

# Load the adapter configuration
peft_model_id = '/kaggle/input/meta-llama-sexbias/pytorch/default/1'
adapter_config = PeftConfig.from_pretrained(peft_model_id)

# Load the LoRA adapter
model = PeftModel.from_pretrained(base_model, peft_model_id, config=adapter_config)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

### Step 7: Test the Fine-tuned Model

Here, we test the base model of `Llama3-8b-fineTuned` on the test set, by the evalation metrics of BLEU and BERTScore.

#### generate Predictions

In [10]:
import re

def generate_predictions(model, tokenizer, dataset):
    predictions = []
    references = []

    model.eval()
    with torch.no_grad():
        for sample in dataset:
            model_input = tokenizer(sample["prompt"], return_tensors="pt").to("cuda")
            output = model.generate(**model_input, max_new_tokens=100)
            prediction = tokenizer.decode(output[0], skip_special_tokens=True)
            
            # Extract the part after "متن بدون بایاس جنسیتی:"
            prediction = prediction.split('\n________\nمتن بدون بایاس جنسیتی:\n')[1].strip()
            prediction = re.sub('متن بدون بایاس جنسیتی:', ' '  , prediction)
            # Remove non-Alphabetical characters and digits execpt dot
            prediction = re.sub(r'[^.آ-یA-Za-z\s]', ' ', prediction)
            # Define a regular expression pattern that matches one or more spaces
            pattern = re.compile(r" +")
            # Apply the pattern to the text and replace the matches with a single space
            prediction = pattern.sub(" ", prediction)
            
            predictions.append(prediction)
            references.append(sample["summary"])
            
            # Clear CUDA cache to free up memory
            torch.cuda.empty_cache()

    return predictions, references

# Generate predictions for base model
tuned_predictions, references = generate_predictions(model, tokenizer, test_dataset)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Save the predictions and references for test set.

In [11]:
with open("tuned_predictions.txt", "w") as f:
    for pred in tuned_predictions:
        f.write(str(pred) +"\n")
        
with open("references.txt", "w") as f:
    for ref in references:
        f.write(str(ref) +"\n")

#### Compute the Evaluation Metrics

Load the predictions and references for test set.

**_Note:_** Don't run this cell untill you faced CUDA OutOfMemory Error.

In [ ]:
references = []
with open("references.txt", "r") as f:
  for line in f:
    references.append(int(line.strip()))
    
tuned_predictions = []
with open("tuned_predictions.txt", "r") as f:
  for line in f:
    tuned_predictions.append(int(line.strip()))

In [19]:
from evaluate import load
from hazm import word_tokenize

bertscore_metric = load("bertscore")
bleu_metric = load('bleu')

def compute_metrics(predictions, references):
    # fix the references with the BLEU score HuggingFace format
    references = [[ref] for ref in references]

    # Compute BLEU score
    bleu_score = bleu_metric.compute(predictions=predictions, references=references, tokenizer=word_tokenize)#["bleu"]

    # Compute BERTScore
    bertscore = bertscore_metric.compute(predictions=predictions, references=references, lang="fa", model_type='microsoft/deberta-v2-xxlarge-mnli')
    bertscore_f1 = sum(bertscore["f1"]) / len(bertscore["f1"])

    return bleu_score, bertscore_f1

# Compute metrics for fine-tuned model
tuned_bleu, tuned_bertscore = compute_metrics(tuned_predictions, references)

print(f"Fine-tuned Model - BLEU: {tuned_bleu}, BERTScore F1: {tuned_bertscore}")

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

Fine-tuned Model - BLEU: {'bleu': 0.4282493906811996, 'precisions': [0.6205770277626566, 0.46599645180366644, 0.37928802588996763, 0.30664760543245173], 'brevity_penalty': 1.0, 'length_ratio': 1.1086300543150271, 'translation_length': 1837, 'reference_length': 1657}, BERTScore F1: 0.9234269032739613


#### Sample Predictions
Run the fine-tuned model on some example inputs:

In [12]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زن‌ها در ورزش‌های سنگین موفق نمی‌شوند
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زن‌ها در ورزش‌های سنگین موفق نمی‌شوند
________
متن بدون بایاس جنسیتی:
موفقیت در ورزش‌های سنگین به توانایی‌های فیزیکی و ذهنی فرد بستگی دارد


In [14]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
المیرا اسم یک دختر است
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
المیرا اسم یک دختر است
________
متن بدون بایاس جنسیتی:
المیرا اسم یک دختر است


In [15]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زن‌ها نمی‌توانند به خوبی مردها طراحی کنند.
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زن‌ها نمی‌توانند به خوبی مردها طراحی کنند.
________
متن بدون بایاس جنسیتی:
هرکسی می‌تواند به خوبی طراحی کند.


In [16]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زنان باید سرپرست سازمان‌دهی رویدادهای اجتماعی باشند.
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
زنان باید سرپرست سازمان‌دهی رویدادهای اجتماعی باشند.
________
متن بدون بایاس جنسیتی:
سرپرستی سازماندهی رویدادهای اجتماعی می‌تواند بر اساس توانایی‌های فردی و علاقه‌های شخصی تقسیم شود.


In [17]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
افراد ممکن است به جنبه های مختلف زندگی خود توجه کنند و تمرکز متفاوتی بر روی مهارت های حرفه ای داشته باشند.
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
افراد ممکن است به جنبه های مختلف زندگی خود توجه کنند و تمرکز متفاوتی بر روی مهارت های حرفه ای داشته باشند.
________
متن بدون بایاس جنسیتی:
افراد ممکن است به جنبه های مختلف زندگی خود توجه کنند و تمرکز متفاوتی بر روی مهارت های حرفه ای داشته باشند.


In [18]:
eval_prompt = """
این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
مردان باید در برنامه‌ریزی برای تعطیلات خانوادگی پیش‌قدم باشند.
________
متن بدون بایاس جنسیتی:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
    
torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



این متن را بدون بایاس جنسیتی کن و بایاس جنسیتی آنرا از بین ببر:
مردان باید در برنامه‌ریزی برای تعطیلات خانوادگی پیش‌قدم باشند.
________
متن بدون بایاس جنسیتی:
برنامه‌ریزی برای تعطیلات خانوادگی باید به عهده‌ی فردی باشد که توانایی‌های لازم را دارد.
